In [1]:
import pandas, seaborn, scipy, numpy, matplotlib, collections, sklearn, math

%matplotlib inline
from matplotlib import pyplot

# Constants
library_sizes = {
    'WGS': 3002000000,
    'WES': 50160183,
}
mhc_binding_threshold_affinity = 500



%matplotlib inline
%config InlineBackend.figure_format = 'png'

matplotlib.rc("savefig", dpi=800)
#matplotlib.rc("savefig", dpi=72)
#matplotlib.rc('text', usetex=False)
#reload(c)

pandas.set_option('display.max_rows', 500)
pandas.set_option('display.max_columns', 500)

def print_full(x):
    pandas.set_option('display.max_rows', len(x))
    print(x)
    pandas.reset_option('display.max_rows')
    
def bootstrap(values, statistic=numpy.mean, samples=5000):
    values = pandas.Series(values).dropna()
    if len(values) <= 1:
        return (numpy.nan, numpy.nan)
    values = [statistic(sklearn.utils.resample(values)) for i in range(samples)]
    return (numpy.percentile(values, 5), numpy.percentile(values, 95))

def round_to_n(x, n):
    return round(x, -int(math.floor(math.log10(x))) + (n - 1)) 

def mean_with_errorbars(values, decimals=0):
    pattern = "%%0.%df" % decimals
    bars = bootstrap(values)
    if numpy.nan in bars:
        return pattern % numpy.mean(values)
    if decimals == 0:
        bars = (round_to_n(bars[0], 2), round_to_n(bars[1], 2)) 
    return (pattern + " (" + pattern + " - " + pattern + ")") % ((numpy.mean(values),) + bars)


/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
sources = pandas.read_csv("../data/sources.csv", index_col="source_id")
sources = sources.ix[sources.cohort == "AOCS"]


In [3]:
all_signatures = pandas.read_csv("../data/all_signatures.csv", index_col=0)
all_signatures.columns = [c.replace("[", "(").replace("]", ")") for c in all_signatures.columns]
all_signatures.index = [
    x.replace("-", " ").replace(":", " ").replace(".", " ").replace(";", " ").replace("  ", " ")
    for x in all_signatures.index
]
all_signatures

,A(C>A)A,A(C>A)C,A(C>A)G,A(C>A)T,A(C>G)A,A(C>G)C,A(C>G)G,A(C>G)T,A(C>T)A,A(C>T)C,A(C>T)G,A(C>T)T,A(T>A)A,A(T>A)C,A(T>A)G,A(T>A)T,A(T>C)A,A(T>C)C,A(T>C)G,A(T>C)T,A(T>G)A,A(T>G)C,A(T>G)G,A(T>G)T,C(C>A)A,C(C>A)C,C(C>A)G,C(C>A)T,C(C>G)A,C(C>G)C,C(C>G)G,C(C>G)T,C(C>T)A,C(C>T)C,C(C>T)G,C(C>T)T,C(T>A)A,C(T>A)C,C(T>A)G,C(T>A)T,C(T>C)A,C(T>C)C,C(T>C)G,C(T>C)T,C(T>G)A,C(T>G)C,C(T>G)G,C(T>G)T,G(C>A)A,G(C>A)C,G(C>A)G,G(C>A)T,G(C>G)A,G(C>G)C,G(C>G)G,G(C>G)T,G(C>T)A,G(C>T)C,G(C>T)G,G(C>T)T,G(T>A)A,G(T>A)C,G(T>A)G,G(T>A)T,G(T>C)A,G(T>C)C,G(T>C)G,G(T>C)T,G(T>G)A,G(T>G)C,G(T>G)G,G(T>G)T,T(C>A)A,T(C>A)C,T(C>A)G,T(C>A)T,T(C>G)A,T(C>G)C,T(C>G)G,T(C>G)T,T(C>T)A,T(C>T)C,T(C>T)G,T(C>T)T,T(T>A)A,T(T>A)C,T(T>A)G,T(T>A)T,T(T>C)A,T(T>C)C,T(T>C)G,T(T>C)T,T(T>G)A,T(T>G)C,T(T>G)G,T(T>G)T
Signature 1,0.011098,0.009149,1.490070e-03,0.006234,0.001801,0.002581,0.000593,0.002964,0.029515,0.014323,0.171647,0.012624,4.021520e-03,0.002371,0.002811,0.008361,0.013916,0.006275,0.010138,0.009256,0.001588,0.001784,0.001386,0.003159,0.006596,0.007342,0.000893,0.007187,0.001285,0.000702,0.000506,0.001382,0.020896,0.018502,0.095577,0.017113,0.001183,0.001903,0.001488,2.179344e-03,0.004177,0.005253,0.007013,6.713813e-03,0.000303,0.002099,0.001600,0.002759,0.008233,0.005758,0.000616,0.004459,0.000602,0.002393,2.485340e-07,0.000890,0.024944,0.027161,0.103571,0.017690,0.000689,0.000552,0.001200,0.002107,0.011248,0.007000,4.977593e-03,0.010667,0.000099,2.023656e-04,0.001188,0.000801,0.012250,0.011162,0.002275,0.015259,0.001875,0.002067,3.048970e-04,0.003152,0.014492,0.017681,0.076002,0.013762,0.005600,0.001999,0.001090,0.003981,0.008074,0.004857,0.008325,0.006257,1.397554e-03,0.001292,2.031077e-03,0.004030
Signature 2,0.000683,0.000619,9.927896e-05,0.000324,0.000263,0.000270,0.000219,0.000611,0.007442,0.002726,0.003322,0.003327,1.323770e-07,0.000113,0.000533,0.000149,0.001304,0.000426,0.000575,0.001488,0.000034,0.000025,0.000273,0.000218,0.000677,0.000214,0.000007,0.000416,0.000028,0.000280,0.000019,0.000313,0.015020,0.003517,0.004979,0.008957,0.000155,0.000464,0.000230,5.748856e-04,0.000547,0.000392,0.000362,5.609001e-04,0.000114,0.000022,0.000228,0.000067,0.000352,0.000134,0.000178,0.000123,0.000045,0.000015,4.066589e-05,0.000268,0.006391,0.001996,0.000303,0.003266,0.000115,0.000294,0.000089,0.000216,0.000005,0.000186,4.950440e-07,0.000579,0.000096,4.700238e-05,0.000110,0.000086,0.015127,0.006532,0.001656,0.012395,0.037242,0.000019,1.625465e-03,0.066880,0.419941,0.081972,0.047720,0.228675,0.000081,0.000005,0.000067,0.000276,0.000102,0.000470,0.000192,0.000585,7.173695e-05,0.000014,2.066152e-04,0.000024
Signature 3,0.022172,0.017872,2.138340e-03,0.016265,0.024003,0.012160,0.005275,0.023278,0.017872,0.008896,0.003573,0.014798,8.428564e-03,0.007373,0.007357,0.008754,0.013036,0.009186,0.011717,0.016979,0.002351,0.001464,0.009054,0.007031,0.018782,0.015760,0.001963,0.014723,0.016833,0.013531,0.004176,0.024046,0.014395,0.008545,0.003518,0.016076,0.007571,0.012725,0.011509,1.645618e-02,0.007895,0.014431,0.008423,1.193243e-02,0.001974,0.005824,0.010465,0.008724,0.009697,0.010843,0.000929,0.012215,0.011917,0.009824,1.671054e-03,0.017914,0.016127,0.008209,0.001213,0.010612,0.004435,0.005615,0.008070,0.008679,0.006850,0.006261,6.098763e-03,0.007509,0.004144,4.501985e-03,0.016391,0.007067,0.011653,0.016607,0.001357,0.016328,0.016041,0.020150,2.527911e-03,0.032674,0.008880,0.013530,0.001705,0.010304,0.007133,0.009103,0.006566,0.014712,0.009115,0.010954,0.006113,0.010774,5.427184e-03,0.006160,1.107653e-02,0.013001
Signature 4,0.036500,0.030900,1.830000e-02,0.024300,0.009700,0.005400,0.003100,0.005400,0.012000,0.007500,0.002800,0.005900,4.800000e-03,0.003900,0.010000,0.003000,0.008400,0.002000,0.008100,0.003600,0.000000,0.000200,0.001500,0.000200,0.046100,0.061400,0.008800,0.043200,0.010500,0.009700,0.006300,0.009400,0.021000,0.014400,0.007600,0.020100,0.007500,0.011100,0.034200,1.150000e-02,0.005200,0.002600,0.010000,5.400000e-03,0.000000,0.001300,0.004600,0.001200,0.037600,0.039900,0.022

In [4]:
deconvolution_df = pandas.read_csv("../data/deconstructsigs_output.cleaned.csv",
                                   index_col=["source_id", "kind"])
for col in deconvolution_df.columns:
    if col.endswith(".1"):
        deconvolution_df[col[:-2]] = deconvolution_df[col]
        del deconvolution_df[col]
deconvolution_df

,,Signature 1,Signature 2,Signature 3,Signature 4,Signature 5,Signature 6,Signature 8,Signature 9,Signature 10,Signature 12,Signature 13,Signature 14,Signature 16,Signature 18,Signature 19,Signature 22,Signature 23,Signature 25,Signature 26,Signature 29,Cisplatin Gallus gallus,Cyclophosphamide Gallus gallus,Etoposide Gallus gallus,Cisplatin C Elegans fcd 2,Cisplatin C Elegans polq 1,Cisplatin C Elegans xpf 1,treated,kind,source_id
source_id,kind,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
AOCS-166-1/SP102133,all,0.000000,0.000000,0.000000,0.000000,0.000000,0.063701,0.000000,0.250905,0.000000,0.514761,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.094825,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,all,AOCS-166-1/SP102133
AOCS-092-13/SP101630,new,0.000000,0.000000,0.133002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.094217,0.120342,0.141969,0.000000,0.000000,0.000000,0.080833,0.152453,0.000000,0.000000,0.000000,0.145093,True,new,AOCS-092-13/SP101630
AOCS-119-13/SP101732,all,0.000000,0.000000,0.217817,0.000000,0.000000,0.000000,0.473673,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.071525,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,all,AOCS-119-13/SP101732
AOCS-117-13/SP101724,all,0.000000,0.000000,0.243353,0.000000,0.000000,0.000000,0.310672,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.067353,0.000000,0.000000,0.000000,0.000000,True,all,AOCS-117-13/SP101724
AOCS-135-13/SP101855,all,0.000000,0.000000,0.272220,0.000000,0.103954,0.000000,0.397365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,all,AOCS-135-13/SP101855
AOCS-135-14/SP101860,all,0.000000,0.000000,0.276793,0.000000,0.110276,0.000000,0.386780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True,all,AOCS-135-14/SP101860
AOCS-150-13/SP102010,all,0.000000,0.000000,0.293942,0.000000,0.000000,0.000000,0.239073,0.000000,0.000000,0.000000,0.000000,0.000000,0.078714,0.00000,0.069938,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074023,0.000000,0.000000,True,all,AOCS-150-13/SP102010
AOCS-004-1/SP101519,all,0.000000,0.000000,0.325053,0.000000,0.176914,0.000000,0.134828,0.000000,0.000000,0.000000,0.000000,0.000000,0.088918,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,all,AOCS-004-1/SP101519
AOCS-088-1/SP101610,all,0.000000,0.000000,0.331926,0.000000,0.000000,0.000000,0.437654,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False,all,AOCS-088-1/SP101610


In [5]:
deconvolved_signatures_all = deconvolution_df.ix[deconvolution_df.kind == "all"].copy()
del deconvolved_signatures_all["treated"]
del deconvolved_signatures_all["kind"]
del deconvolved_signatures_all["source_id"]
deconvolved_signatures_all.index = deconvolved_signatures_all.index.droplevel(1)
deconvolved_signatures_all.shape

(115, 26)

In [6]:
deconvolved_signatures_new = deconvolution_df.ix[deconvolution_df.kind == "new"]
del deconvolved_signatures_new["treated"]
del deconvolved_signatures_new["kind"]
del deconvolved_signatures_new["source_id"]

deconvolved_signatures_new.index = deconvolved_signatures_new.index.droplevel(1)
deconvolved_signatures_new.shape

(14, 26)

In [7]:
mutations = pandas.read_csv("../data/mutations.csv.bz2")
mutations["indel"] = mutations.ref.str.len() != mutations.alt.str.len()


/Users/tim/venvs/analysis-venv-2.7/lib/python2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [9]:
all_mutations_with_signatures = mutations.ix[mutations.snv]
new_mutations_with_signatures = mutations.ix[
    mutations.snv & mutations.source_id.isin(deconvolved_signatures_new.index) & mutations.unique_to_treated
]

def add_signature_columns(mutations_df, signature_participations, all_signatures):
    mutations_df = mutations_df.copy().reset_index()
    extra_columns = []
    for (i, row) in mutations_df.iterrows():
        unnormalized = numpy.array([
            all_signatures.ix[signature][row.context_mutation_3p5p] * participation
            for (signature, participation) in signature_participations.ix[row.source_id].iteritems()
        ])
        if numpy.isnan(unnormalized).any():
            print(i, row, unnormalized, signature, row.context_mutation_3p5, participation)
            raise ValueError()
        normalized = unnormalized / unnormalized.sum()
        extra_columns.append(normalized)
    new_df = pandas.DataFrame(extra_columns, columns=signature_participations.columns)
    for col in new_df.columns:
        mutations_df[col] = new_df[col]
    return mutations_df


all_mutations_with_signatures = add_signature_columns(
    all_mutations_with_signatures, deconvolved_signatures_all, all_signatures)
all_mutations_with_signatures

new_mutations_with_signatures = add_signature_columns(
    new_mutations_with_signatures, deconvolved_signatures_new, all_signatures)
new_mutations_with_signatures
            

KeyError: 'TCGA-1705-02A'

In [ ]:
all_mutations_with_signatures

In [ ]:
'''
mutations_with_signatures = pandas.concat([
        primary_mutations_with_signatures.ix[sources.index[~sources.treated]],
        new_mutations_with_signatures,
], ignore_index=True)
print(mutations_with_signatures.shape)
mutations_with_signatures.to_csv("../data/mutations_with_signatures.csv")
mutations_with_signatures
'''

In [ ]:
muts_confident = mutations.ix[
    (mutations.called == 1) | (mutations.dna_alt_reads > 10)
]
muts_confident_coding = muts_confident.ix[muts_confident.effect.str.startswith("p.")]
muts_confident_coding_expressed = muts_confident_coding.ix[muts_confident_coding.rna_alt_reads > 2]

non_exon_effects = [
    "intergenic",
    "intragenic",
    "intronic",
    "non-coding-transcript",
    "incomplete",
    "splice-acceptor",
    "splice-donor",
    "intronic-splice-site",
    "exonic-splice-site",
    "3' UTR",
    "5' UTR",
]

muts_confident_exon = muts_confident.ix[~ muts_confident.effect.isin(non_exon_effects)]
muts_exon = mutations.ix[~ mutations.effect.isin(non_exon_effects)]

[x for x in muts_confident_exon.effect.unique().tolist() if not x.startswith("p.")]


In [ ]:
display_signatures = ["Signature 1", "Signature 3", "Signature 8"]

sources_with_signature_counts = sources.copy()
for source_id, grouped in all_mutations_with_signatures.groupby("source_id"):
    if sources.ix[source_id].treated:
        sub_all_muts = mutations.ix[(mutations.source_id == source_id) & mutations.unique_to_treated]
        sub_exon_muts = muts_exon.ix[(mutations.source_id == source_id) & mutations.unique_to_treated]
    else:
        sub_all_muts = mutations.ix[(mutations.source_id == source_id)]
        sub_exon_muts = muts_exon.ix[(mutations.source_id == source_id)]
        
    num_mutations = float(sub_all_muts.shape[0])
    num_exon_mutations = float(sub_exon_muts.shape[0])
    num_neoantigens = float(sub_all_muts.num_binders.sum())
    
    snv_mutations_accounted_for = 0.0
    snv_neoantigens_accounted_for = 0.0
    for sig in display_signatures:
        snv_mutations_accounted_for += grouped[sig].sum()
        snv_neoantigens_accounted_for += numpy.dot(grouped[sig], grouped.num_binders)
        
        sources_with_signature_counts.loc[source_id, "%s mutations" % sig] = grouped[sig].sum() / num_mutations
        sources_with_signature_counts.loc[source_id, "%s neoantigens" % sig] = (
            numpy.dot(grouped[sig], grouped.num_binders)) / num_neoantigens
    
    snv_mutations = sub_all_muts.snv.sum()
    snv_neoantigens = sub_all_muts.ix[sub_all_muts.snv].num_binders.sum()
    
    sources_with_signature_counts.loc[source_id, "other snv mutations"] = (
        snv_mutations - snv_mutations_accounted_for) / num_mutations
    sources_with_signature_counts.loc[source_id, "other snv neoantigens"] = (
        snv_neoantigens - snv_neoantigens_accounted_for) / num_neoantigens

    sources_with_signature_counts.loc[source_id, "snv mutations"] = snv_mutations
    sources_with_signature_counts.loc[source_id, "snv neoantigens"] = snv_neoantigens
    
    sources_with_signature_counts.loc[source_id, "mnv mutations"] = (
        (~ sub_all_muts.indel) & (~ sub_all_muts.snv)).sum() / num_mutations
    sources_with_signature_counts.loc[source_id, "mnv neoantigens"] = (
        sub_all_muts.ix[(~ sub_all_muts.indel) & (~ sub_all_muts.snv)].num_binders.sum()) / num_neoantigens
    
    sources_with_signature_counts.loc[source_id, "indel mutations"] = (
        (sub_all_muts.indel).sum()) / num_exon_mutations
    sources_with_signature_counts.loc[source_id, "indel neoantigens"] = (
        sub_all_muts.ix[sub_all_muts.indel].num_binders.sum()) / num_neoantigens
    
sources_with_signature_counts[sources_with_signature_counts.paired]
    